### Домашнее задание. 

## Урок 1. Инсталлы. Стартовая воронка
#### Выполнил работу: Левин Алексей

## Домашнее задание

+ 1. Проанализируйте динамику количества ежедневных инсталлов в игре для
органики и неорганики. Какие особенности изменений этих показателей вы
видите? Сгенерируйте гипотезы, почему могут наблюдаться такие изменения
(которые затем в ходе выполнения дальнейших заданий вы сможете проверить).
+ 2. Есть ли отличия этого показателя для разных регионов (СНГ, Европа, Азия,
Северная Америка + Канада, Южная Америка)? Сделайте оценку, в каких регионах
более качественное удержание на начальном этапе (можно оценить, например, по
второй сессии и проценту игроков, запустивших игру повторно).
+ 3. Проанализируйте поведение игроков в первые 24 часа после первого запуска игры.
Какие события совершают пользователи чаще всего? Есть ли паттерны поведения
игроков разных когорт (по типу пользователей, регионам и т.д.)?

# Введение (подключение к библиотекам и бд)

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import IFrame
plt.style.use('seaborn-pastel')
pio.renderers.default='iframe'

In [ ]:
# Данные учебной базы сохраним в переменные
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'data_viz_1068.project_dataset'

# Импортируем create_engine из библиотеки sqlalchemy
from sqlalchemy import create_engine

#Пропишем функцию engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



## 1. Проанализируйте динамику количества ежедневных инсталлов в игре для органики и неорганики. Какие особенности изменений этих показателей вы видите? Сгенерируйте гипотезы, почему могут наблюдаться такие изменения (которые затем в ходе выполнения дальнейших заданий вы сможете проверить).

# 1. Проанализируйте динамику количества ежедневных инсталлов в игре для органики и неорганики. Какие особенности изменений этих показателей вы видите? Сгенерируйте гипотезы, почему могут наблюдаться такие изменения (которые затем в ходе выполнения дальнейших заданий вы сможете проверить).

In [ ]:
# вызовем таблицу
df_project_dataset = pd.read_sql(f'''
SELECT * FROM {TABLE_SCHEMA}
LIMIT 10
''', con=engine)
df_project_dataset

,event_time,event_date,event_name,revenue_usd,region,country,device_type,platform,cohort_date,user_id,user_type,content_id,currency,revenue
0,2021-04-02 12:27:57,2021-04-02,LaunchApp,0.0,EU,RU,HONOR-STK-LX1,android,2021-02-01,150585,organic,None,None,0.0
1,2021-04-02 12:28:47,2021-04-02,LaunchApp,0.0,EU,RU,iPhone 6,ios,2020-10-29,179696,non_organic,None,None,0.0
2,2021-04-02 12:28:48,2021-04-02,LaunchApp,0.0,EU,UA,Redmi-Redmi Note 9 Pro,android,2021-03-26,226754,organic,None,None,0.0
3,2021-04-02 12:30:19,2021-04-02,LaunchApp,0.0,EU,RU,Xiaomi-MI 9,android,2021-03-24,227448,organic,None,None,0.0
4,2021-04-02 12:30:36,2021-04-02,LaunchApp,0.0,EU,RU,HONOR-COL-L29,android,2020-10-09,139354,non_organic,None,None,0.0
5,2021-04-02 12:31:29,2021-04-02,LaunchApp,0.0,EU,RU,samsung-SM-A105F,android,2020-11-09,196878,organic,None,None,0.0
6,2021-04-02 12:31:35,2021-04-02,LaunchApp,0.0,EU,RU,HUAWEI-EVA-L19,android,2021-03-30,227555,organic,None,None,0.0
7,2021-04-02 12:31:36,2021-04-02,LaunchApp,0.0,EU,RU,xiaomi-Redmi 6,android,2020-10-26,171065,non_organic,None,None,0.0
8,2021-04-02 12:31:53,2021-04-02,LaunchApp,0.0,EU,RU,samsung-SM-A505FM,android,2021-03-21,224822,organic,None,None,0.0
9,2021-04-02 12:31:56,2021-04-02,LaunchApp,0.0,EU,RU,TECNO-TECNO KD6,android,2020-10-08,136995,non_organic,None,None,0.0


In [ ]:
# Оставим только столбцы event_date, region,	country,	device_type,	platform, cohort_date, user_type, user_id с параметром "первого запуска" (FirstLaunchApp)
df1 = pd.read_sql(f'''
    SELECT
    cast(event_date as date), region,	country,	device_type,	platform, cohort_date, user_type, user_id
      FROM {TABLE_SCHEMA}
    WHERE
        event_name ='FirstLaunchApp'
    ORDER BY event_date
''', con=engine)
df1

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2020-10-22,EU,RU,Xiaomi-Redmi Note 4,android,2020-10-22,organic,100206
1,2020-10-22,AS,KR,samsung-SM-N950N,android,2020-10-22,organic,100024
2,2020-10-22,AS,KR,iPad Air (3rd generation),ios,2020-10-22,organic,100025
3,2020-10-22,AS,KR,samsung-SM-N950N,android,2020-10-22,organic,100026
4,2020-10-22,AS,KR,samsung-SM-N950N,android,2020-10-22,organic,100027
...,...,...,...,...,...,...,...,...
51584,2021-05-31,AS,KZ,xiaomi-Redmi Note 8,android,2021-04-22,organic,215899
51585,2021-05-31,EU,RU,HUAWEI-POT-LX1,android,2021-05-31,organic,137125
51586,2021-05-31,EU,RU,iPhone 6,ios,2021-05-31,organic,138521
51587,2021-05-31,EU,RU,Redmi-Redmi Note 8 Pro,android,2020-11-06,organic,232522


In [ ]:
sum_install = df1.groupby(['user_type', 'event_date'], as_index = False)['user_id'].count().rename(columns = {'user_id':'sum_user'})
sum_install

,user_type,event_date,sum_user
0,non_organic,2020-10-27,153
1,non_organic,2020-10-28,353
2,non_organic,2020-10-29,354
3,non_organic,2020-10-30,262
4,non_organic,2020-10-31,124
...,...,...,...
424,organic,2021-05-27,82
425,organic,2021-05-28,88
426,organic,2021-05-29,96
427,organic,2021-05-30,120


# Визуализация первой задачи, гипотезы

In [ ]:
# Визуализируем Органический и НЕ органический траффики по кол-ву install
fig1 = px.bar(sum_install, x='event_date', y='sum_user', color = 'user_type', title='Органический и Неорганический траффики по сумме инсталлов')
fig1.show(renderer='colab')

С ноября по начало декабря наблюдаются скачки. С начала декабря ещё был небольшой скачок (на 126 новых игроков), но после этого случая, число пользователей устанавливающих приложение из сторонних источников - колеблется в пределах 5-20.
Органический трафик тем временем подскочил на 796 установок 23 марта.
Посмотрим на органику в разрезе платформ, чтобы понять, где был скачок

In [ ]:
# Посмотрим Органику в разрезе платформ
install_platform = df1[df1['user_type'] == 'organic']
install_platform = install_platform.groupby(['platform', 'event_date'], as_index = False)['user_id'].count().rename(columns = {'user_id':'sum_user'})
# Визуализируем
fig2 = px.bar(install_platform, x='event_date', y='sum_user', color = 'platform', title='Органический траффик по сумме инсталлов (в разрезе платформ)')
fig2.show(renderer='colab')

Итак, как мы видим, скачок органического трафика в марте был только на устройствах андроид.



# 2.1. Есть ли отличия этого показателя для разных регионов (СНГ, Европа, Азия, Северная Америка + Канада, Южная Америка)?

Рассчитываем данные

In [ ]:
# СНГ
df_reg_cis = pd.read_sql(f'''
SELECT
    cast(event_date as date), region,	country,	device_type,	platform, cohort_date, user_type, user_id
      FROM {TABLE_SCHEMA}
    WHERE
        event_name ='FirstLaunchApp'
        and country in ('RU', 'KZ', 'AZ', 'AM', 'BY', 'KG', 'MD', 'TJ', 'UZ', 'UA')
''', con=engine)
df_reg_cis

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,EU,RU,samsung-SM-J810F,android,2021-04-02,organic,227866
1,2021-04-02,EU,RU,iPhone 11,ios,2021-04-02,organic,227867
2,2021-04-02,EU,RU,HONOR-JAT-LX1,android,2021-04-02,organic,130251
3,2021-04-02,EU,RU,iPhone 11,ios,2021-04-02,organic,127643
4,2021-04-02,EU,UA,iPhone 11,ios,2021-04-02,organic,227869
...,...,...,...,...,...,...,...,...
34265,2021-04-02,EU,BY,Redmi-Redmi Note 9 Pro,android,2020-12-16,organic,213809
34266,2021-04-02,EU,RU,xiaomi-Redmi Note 7,android,2021-04-02,organic,227864
34267,2021-04-02,EU,BY,Xiaomi-Redmi 8,android,2021-03-20,organic,225363
34268,2021-04-02,EU,UA,Xiaomi-Redmi 5,android,2021-03-12,organic,220180


In [ ]:
# Создадим отдельную группу CIS для стран СНГ
df_reg_cis['region']='CIS'
df_reg_cis

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,CIS,RU,samsung-SM-J810F,android,2021-04-02,organic,227866
1,2021-04-02,CIS,RU,iPhone 11,ios,2021-04-02,organic,227867
2,2021-04-02,CIS,RU,HONOR-JAT-LX1,android,2021-04-02,organic,130251
3,2021-04-02,CIS,RU,iPhone 11,ios,2021-04-02,organic,127643
4,2021-04-02,CIS,UA,iPhone 11,ios,2021-04-02,organic,227869
...,...,...,...,...,...,...,...,...
34265,2021-04-02,CIS,BY,Redmi-Redmi Note 9 Pro,android,2020-12-16,organic,213809
34266,2021-04-02,CIS,RU,xiaomi-Redmi Note 7,android,2021-04-02,organic,227864
34267,2021-04-02,CIS,BY,Xiaomi-Redmi 8,android,2021-03-20,organic,225363
34268,2021-04-02,CIS,UA,Xiaomi-Redmi 5,android,2021-03-12,organic,220180


In [ ]:
# Без СНГ
df_reg_other = pd.read_sql(f'''
SELECT
    cast(event_date as date), region,	country,	device_type,	platform, cohort_date, user_type, user_id
      FROM {TABLE_SCHEMA}
    WHERE
        event_name ='FirstLaunchApp'
        and country not in ('RU', 'KZ', 'AZ', 'AM', 'BY', 'KG', 'MD', 'TJ', 'UZ', 'UA')
''', con=engine)
df_reg_other

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,EU,LT,HUAWEI-EML-L29,android,2021-04-02,organic,227865
1,2021-04-02,AS,JP,iPhone XR,ios,2021-04-02,organic,204565
2,2021-04-02,AS,KR,samsung-SM-G988N,android,2021-01-29,non_organic,219432
3,2021-04-02,EU,DE,Xiaomi-Redmi 8,android,2021-02-15,organic,218773
4,2021-04-02,AS,KR,samsung-SM-N950N,android,2021-01-30,organic,220700
...,...,...,...,...,...,...,...,...
17312,2021-04-02,AS,KR,samsung-SM-G977N,android,2021-01-29,organic,219469
17313,2021-04-02,EU,TR,Sony-G8341,android,2021-04-02,organic,227846
17314,2021-04-02,EU,LT,Lenovo-Lenovo TB-7304X,android,2021-03-18,organic,226968
17315,2021-04-02,EU,EE,Xiaomi-Mi 10 Pro,android,2021-04-02,organic,227859


In [ ]:
frames = [df_reg_cis, df_reg_other]
reg = pd.concat(frames)
reg

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,CIS,RU,samsung-SM-J810F,android,2021-04-02,organic,227866
1,2021-04-02,CIS,RU,iPhone 11,ios,2021-04-02,organic,227867
2,2021-04-02,CIS,RU,HONOR-JAT-LX1,android,2021-04-02,organic,130251
3,2021-04-02,CIS,RU,iPhone 11,ios,2021-04-02,organic,127643
4,2021-04-02,CIS,UA,iPhone 11,ios,2021-04-02,organic,227869
...,...,...,...,...,...,...,...,...
17312,2021-04-02,AS,KR,samsung-SM-G977N,android,2021-01-29,organic,219469
17313,2021-04-02,EU,TR,Sony-G8341,android,2021-04-02,organic,227846
17314,2021-04-02,EU,LT,Lenovo-Lenovo TB-7304X,android,2021-03-18,organic,226968
17315,2021-04-02,EU,EE,Xiaomi-Mi 10 Pro,android,2021-04-02,organic,227859


In [ ]:
# Считаем кол-во установок по регионам
reg_sum_install = reg.groupby(['region', 'event_date'], as_index = False)['user_id'].count().rename(columns = {'user_id':'sum_user'})
reg_sum_install

,region,event_date,sum_user
0,AF,2020-11-03,1
1,AF,2020-11-05,1
2,AF,2020-11-07,1
3,AF,2020-11-13,1
4,AF,2020-11-19,1
...,...,...,...
960,SA,2021-04-11,2
961,SA,2021-04-13,1
962,SA,2021-05-04,1
963,SA,2021-05-06,1


# Визуализация Второго задания 1-й график



In [ ]:
# Визуализируем
fig3 = px.line(reg_sum_install, x='event_date', y='sum_user', color = 'region', markers=True, title='Кол-во установок по регионам')
fig3.show(renderer='colab')

Скачок в марте произошёл только по Регионам CIS ('RU', 'KZ', 'AZ', 'AM', 'BY', 'KG', 'MD', 'TJ', 'UZ', 'UA'), т.е. была попытка поработать с русскоязычными блогерами, упоминувшие игру в своих каналах. Они правлекли дешёвых и не слишком заинтересованных игроков (пользователи надолго в игре не задерживались).
Сравнивая с предыдущими графиками - это органические пользователи андроид, значит ссылка, которой поделились блоггеры была на google play.

# 2.2.В каких регионах более качественное удержание на начальном этапе (можно оценить, например, по второй сессии и проценту игроков, запустивших игру повторно)?

In [ ]:
#Считаем кол-во логинов (LaunchApp) для CIS
df_log_reg_cis = pd.read_sql(f'''
SELECT
    cast(event_date as date), region,	country,	device_type,	platform, cohort_date, user_type, user_id
      FROM {TABLE_SCHEMA}
    WHERE
        event_name ='LaunchApp'
        and country in ('RU', 'KZ', 'AZ', 'AM', 'BY', 'KG', 'MD', 'TJ', 'UZ', 'UA')
''', con=engine)
df_log_reg_cis

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,EU,RU,HONOR-STK-LX1,android,2021-02-01,organic,150585
1,2021-04-02,EU,RU,iPhone 6,ios,2020-10-29,non_organic,179696
2,2021-04-02,EU,UA,Redmi-Redmi Note 9 Pro,android,2021-03-26,organic,226754
3,2021-04-02,EU,RU,Xiaomi-MI 9,android,2021-03-24,organic,227448
4,2021-04-02,EU,RU,HONOR-COL-L29,android,2020-10-09,non_organic,139354
...,...,...,...,...,...,...,...,...
605015,2021-04-02,EU,UA,iPhone 6s,ios,2021-03-11,organic,215505
605016,2021-04-02,EU,RU,vivo-V2027,android,2020-12-21,organic,214687
605017,2021-04-02,EU,RU,Xiaomi-POCOPHONE F1,android,2021-03-22,organic,225154
605018,2021-04-02,EU,UA,Redmi-Redmi Note 9 Pro,android,2020-12-14,organic,101925


In [ ]:
df_log_reg_cis['region']='CIS'
df_log_reg_cis

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,CIS,RU,HONOR-STK-LX1,android,2021-02-01,organic,150585
1,2021-04-02,CIS,RU,iPhone 6,ios,2020-10-29,non_organic,179696
2,2021-04-02,CIS,UA,Redmi-Redmi Note 9 Pro,android,2021-03-26,organic,226754
3,2021-04-02,CIS,RU,Xiaomi-MI 9,android,2021-03-24,organic,227448
4,2021-04-02,CIS,RU,HONOR-COL-L29,android,2020-10-09,non_organic,139354
...,...,...,...,...,...,...,...,...
605015,2021-04-02,CIS,UA,iPhone 6s,ios,2021-03-11,organic,215505
605016,2021-04-02,CIS,RU,vivo-V2027,android,2020-12-21,organic,214687
605017,2021-04-02,CIS,RU,Xiaomi-POCOPHONE F1,android,2021-03-22,organic,225154
605018,2021-04-02,CIS,UA,Redmi-Redmi Note 9 Pro,android,2020-12-14,organic,101925


In [ ]:
# Без СНГ
df_log_reg_other = pd.read_sql(f'''
SELECT
    cast(event_date as date), region,	country,	device_type,	platform, cohort_date, user_type, user_id
      FROM {TABLE_SCHEMA}
    WHERE
        event_name ='LaunchApp'
        and country not in ('RU', 'KZ', 'AZ', 'AM', 'BY', 'KG', 'MD', 'TJ', 'UZ', 'UA')
''', con=engine)
df_log_reg_other

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,AS,KR,samsung-SM-G977N,android,2021-01-29,organic,219469
1,2021-04-02,AS,KR,samsung-SM-N971N,android,2021-01-29,organic,219436
2,2021-04-02,AS,KR,samsung-SM-G977N,android,2021-01-30,organic,219510
3,2021-04-02,AS,KR,samsung-SM-G970N,android,2021-01-30,organic,219559
4,2021-04-02,AS,KR,samsung-SM-G988N,android,2021-01-29,non_organic,219834
...,...,...,...,...,...,...,...,...
420292,2021-04-02,AS,KR,samsung-SM-G965N,android,2021-02-02,organic,220052
420293,2021-04-02,AS,KR,samsung-SM-G965N,android,2021-01-19,organic,218363
420294,2021-04-02,AS,KR,samsung-SM-N960N,android,2021-01-30,organic,219520
420295,2021-04-02,EU,PL,samsung-SM-A207F,android,2020-09-05,organic,188152


In [ ]:
frames = [df_log_reg_cis, df_log_reg_other]
log_reg = pd.concat(frames)
log_reg

,event_date,region,country,device_type,platform,cohort_date,user_type,user_id
0,2021-04-02,CIS,RU,HONOR-STK-LX1,android,2021-02-01,organic,150585
1,2021-04-02,CIS,RU,iPhone 6,ios,2020-10-29,non_organic,179696
2,2021-04-02,CIS,UA,Redmi-Redmi Note 9 Pro,android,2021-03-26,organic,226754
3,2021-04-02,CIS,RU,Xiaomi-MI 9,android,2021-03-24,organic,227448
4,2021-04-02,CIS,RU,HONOR-COL-L29,android,2020-10-09,non_organic,139354
...,...,...,...,...,...,...,...,...
420292,2021-04-02,AS,KR,samsung-SM-G965N,android,2021-02-02,organic,220052
420293,2021-04-02,AS,KR,samsung-SM-G965N,android,2021-01-19,organic,218363
420294,2021-04-02,AS,KR,samsung-SM-N960N,android,2021-01-30,organic,219520
420295,2021-04-02,EU,PL,samsung-SM-A207F,android,2020-09-05,organic,188152


In [ ]:
# Оставим в таблице log_reg только столбцы event_date, region и user_id
log_reg.drop(['country', 'device_type', 'platform', 'cohort_date', 'user_type'], axis = 1, inplace = True)
log_reg

,event_date,region,user_id
0,2021-04-02,CIS,150585
1,2021-04-02,CIS,179696
2,2021-04-02,CIS,226754
3,2021-04-02,CIS,227448
4,2021-04-02,CIS,139354
...,...,...,...
420292,2021-04-02,AS,220052
420293,2021-04-02,AS,218363
420294,2021-04-02,AS,219520
420295,2021-04-02,EU,188152


In [ ]:
# Из таблицы, расчитанной в пункте 2.1 (reg) возьмём только столбцы event_date и user_id
inst = reg[['event_date', 'user_id']]
# Переименуем столбец event_date в install_date
inst = inst.rename(columns = {'event_date':'install_date'})
# Сгруппируем по user_id (по возрастанию) выбрав минимальные даты
inst = inst.groupby('user_id', as_index = False)['install_date'].min()
inst

,user_id,install_date
0,100003,2021-01-18
1,100006,2021-02-14
2,100007,2020-12-22
3,100010,2021-01-18
4,100011,2021-01-18
...,...,...
39118,232719,2021-05-31
39119,232720,2021-05-31
39120,232722,2021-05-31
39121,232723,2021-05-31


In [ ]:
# В таблице Log_reg уберём дубликаты для id
log_reg = log_reg.drop_duplicates().reset_index(drop=True)

In [ ]:
# Смержем  созданные таблички log_reg и inst по столбцу user_id
inst_log_reg = log_reg.merge(
    inst,
    how = 'left',
    on = 'user_id'
)
inst_log_reg

,event_date,region,user_id,install_date
0,2021-04-02,CIS,150585,2020-11-19
1,2021-04-02,CIS,179696,2020-10-29
2,2021-04-02,CIS,226754,2021-03-26
3,2021-04-02,CIS,227448,2021-03-30
4,2021-04-02,CIS,139354,NaN
...,...,...,...,...
404575,2021-04-02,AS,221846,2021-02-18
404576,2021-04-02,EU,224538,2021-03-18
404577,2021-04-02,AS,212587,2020-12-13
404578,2021-04-02,AS,217692,2021-01-12


In [ ]:
# По таблице видно, что в install_date присутствуют данные NaN, зададим для них значения из столбца event_date
inst_log_reg['install_date'] = inst_log_reg['install_date'].fillna(inst_log_reg['event_date'])
inst_log_reg

,event_date,region,user_id,install_date
0,2021-04-02,CIS,150585,2020-11-19
1,2021-04-02,CIS,179696,2020-10-29
2,2021-04-02,CIS,226754,2021-03-26
3,2021-04-02,CIS,227448,2021-03-30
4,2021-04-02,CIS,139354,2021-04-02
...,...,...,...,...
404575,2021-04-02,AS,221846,2021-02-18
404576,2021-04-02,EU,224538,2021-03-18
404577,2021-04-02,AS,212587,2020-12-13
404578,2021-04-02,AS,217692,2021-01-12


In [ ]:
inst_log_reg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404580 entries, 0 to 404579
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   event_date    404580 non-null  object
 1   region        404580 non-null  object
 2   user_id       404580 non-null  int64 
 3   install_date  404580 non-null  object
dtypes: int64(1), object(3)
memory usage: 15.4+ MB


In [ ]:
# Меняем формат дат
inst_log_reg['event_date'] = pd.to_datetime(inst_log_reg['event_date'])
inst_log_reg['install_date'] = pd.to_datetime(inst_log_reg['install_date'])

In [ ]:
# Считаем Life-time
inst_log_reg['lifetime'] = inst_log_reg['event_date'] - inst_log_reg['install_date']
inst_log_reg['lifetime'] = inst_log_reg['lifetime']/np.timedelta64(1, 'D')
inst_log_reg['lifetime'] = inst_log_reg['lifetime'].astype('int')

In [ ]:
# Удалим дубли
inst_log_reg = inst_log_reg.drop_duplicates().reset_index(drop=True)
# Удалим отрицательные life-time
inst_log_reg = inst_log_reg[inst_log_reg['lifetime'] >=0]

In [ ]:
# Нам нужны life-time первых 2-х дней
inst_log_reg1_2d = inst_log_reg[(inst_log_reg['lifetime'] >=0) & (inst_log_reg['lifetime'] <=2)]
# Группируем полученные данные по столбцам region и lifetime с уникальными id
inst_log_reg1_2d = inst_log_reg1_2d.groupby(['region', 'lifetime'], as_index = False)['user_id'].nunique()
inst_log_reg1_2d

,region,lifetime,user_id
0,AF,0,21
1,AF,1,8
2,AF,2,10
3,AS,0,1086
4,AS,1,430
5,AS,2,330
6,AU,0,11
7,AU,1,6
8,AU,2,7
9,CIS,0,23965


In [ ]:
# Группируем таблицу inst_log_reg по region с кол-вом уникальных id.
cohort_user_country = inst_log_reg.groupby(['region'], as_index = False)['user_id'].nunique().rename(columns = {'user_id':'sum_users'})
cohort_user_country

,region,sum_users
0,AF,32
1,AS,1306
2,AU,22
3,CIS,27866
4,EU,751
5,NA,13271
6,SA,61


In [ ]:
# Соединим таблицы inst_log_reg1_2d и cohort_user_country (по столбцу region) и назовём новую таблицу retention
retantion = inst_log_reg1_2d.merge(
    cohort_user_country,
    how = 'left',
    on = 'region'
)
retantion

,region,lifetime,user_id,sum_users
0,AF,0,21,32
1,AF,1,8,32
2,AF,2,10,32
3,AS,0,1086,1306
4,AS,1,430,1306
5,AS,2,330,1306
6,AU,0,11,22
7,AU,1,6,22
8,AU,2,7,22
9,CIS,0,23965,27866


In [ ]:
# Переименуем колонки sum_users и user_id на cohort_user_country и user_id_1d, соответственно
retantion = retantion.rename(columns = {'sum_users':'cohort_user_country', 'user_id':'user_id_1d'})
# Посчитаем столбец retention rate разделив столбец user_id_1d на cohort_user_country
retantion['ret_rate'] = round(retantion['user_id_1d']/retantion['cohort_user_country']*100, 2)
# Посчитаем столбец лаги retention rate сгруппировав по столбцу region и сдвинув столбец ret_rate на одну позицию
retantion['lag_ret_rate'] = retantion.groupby(['region'])['ret_rate'].shift(1)
# Все пустые значения в столбце lag_ret_rate заполним данными из столбца ret_rate
retantion['lag_ret_rate'] = retantion['lag_ret_rate'].fillna(retantion['ret_rate'])
# Создадим новый столбец delta (отняв ret_rate от lag_ret_rate )
retantion['delta'] = retantion['lag_ret_rate'] - retantion['ret_rate']
retantion

,region,lifetime,user_id_1d,cohort_user_country,ret_rate,lag_ret_rate,delta
0,AF,0,21,32,65.62,65.62,0.00
1,AF,1,8,32,25.00,65.62,40.62
2,AF,2,10,32,31.25,25.00,-6.25
3,AS,0,1086,1306,83.15,83.15,0.00
4,AS,1,430,1306,32.92,83.15,50.23
5,AS,2,330,1306,25.27,32.92,7.65
6,AU,0,11,22,50.00,50.00,0.00
7,AU,1,6,22,27.27,50.00,22.73
8,AU,2,7,22,31.82,27.27,-4.55
9,CIS,0,23965,27866,86.00,86.00,0.00


# Визуализация Второго задания 2-й и 3-й график

In [ ]:
# Изобразим полученые данные на гистограмме (долю, не зашеджших игроков)
fig4 = px.histogram(retantion, x='region', y='delta', color='lifetime', barmode='group', labels = {'delta':'доля не зашедших'}, title='Доля не зашедших игроков', height=500)
fig4.show(renderer='colab')

In [ ]:
# Изобразим ret_rate (долю, вернувшихся 0,1 и 2 дней)
fig5 = px.histogram(retantion, x='region', y='ret_rate', color='lifetime', barmode='group', title='Retention нулевого, первого и второго дней', height=500)
fig5.show(renderer='colab')

Как видно из графиков- наибьольший отвал игроков первого дня происходил в Италии (NA) - 60,21%. Наибольший процент вернушихся в первый день был в Американское Самоа (AS) - 32,92% и CIS - 31,82%

# 3. Проанализируйте поведение игроков в первые 24 часа после первого запуска игры. Какие события совершают пользователи чаще всего? Есть ли паттерны поведения игроков разных когорт (по типу пользователей, регионам и т.д.)?

In [ ]:
# Вызовем нашу таблицу но с определёнными полями для расчётов (лишние уберём)
df3_1 = pd.read_sql(f'''
SELECT
    distinct event_time, event_name,	revenue_usd, region,	country, platform,	user_id, user_type, content_id
      FROM {TABLE_SCHEMA}
    ORDER BY user_id
''', con=engine)
df3_1

,event_time,event_name,revenue_usd,region,country,platform,user_id,user_type,content_id
0,2020-11-13 13:43:31,LaunchApp,0.0,EU,RU,android,100002,organic,None
1,2020-11-13 13:50:12,LaunchApp,0.0,EU,RU,android,100002,organic,None
2,2020-11-13 13:58:15,LaunchApp,0.0,EU,RU,android,100002,organic,None
3,2020-11-13 14:08:26,LaunchApp,0.0,EU,RU,android,100002,organic,None
4,2020-11-13 14:20:02,LaunchApp,0.0,EU,RU,android,100002,organic,None
...,...,...,...,...,...,...,...,...,...
1176899,2021-05-31 13:19:12,achieve_level_4,0.0,EU,UA,ios,232720,organic,None
1176900,2021-05-31 13:40:42,achieve_level_5,0.0,EU,UA,ios,232720,organic,None
1176901,2021-05-31 14:15:21,FirstLaunchApp,0.0,EU,RU,ios,232722,organic,None
1176902,2021-05-31 14:20:52,FirstLaunchApp,0.0,EU,RU,ios,232723,organic,None


In [ ]:
# Создадим ещё одну таблицу с полями первого логина и id
df_1st_log = pd.read_sql(f'''
SELECT MIN(event_time) as first_log, user_id
FROM {TABLE_SCHEMA}
GROUP BY user_id
ORDER BY user_id
''', con=engine)
df_1st_log

,first_log,user_id
0,2020-11-13 13:43:31,100002
1,2021-01-18 12:12:13,100003
2,2020-12-01 08:44:58,100006
3,2020-12-22 07:53:50,100007
4,2021-01-22 06:13:19,100008
...,...,...
52891,2021-05-31 12:08:01,232719
52892,2021-05-31 12:33:20,232720
52893,2021-05-31 14:15:21,232722
52894,2021-05-31 14:20:52,232723


In [ ]:
# Смержим наши таблички по user_id
df3_2 = df3_1.merge(
  df_1st_log,
  on='user_id',
  how='left'
)
df3_2

,event_time,event_name,revenue_usd,region,country,platform,user_id,user_type,content_id,first_log
0,2020-11-13 13:43:31,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31
1,2020-11-13 13:50:12,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31
2,2020-11-13 13:58:15,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31
3,2020-11-13 14:08:26,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31
4,2020-11-13 14:20:02,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31
...,...,...,...,...,...,...,...,...,...,...
1176899,2021-05-31 13:19:12,achieve_level_4,0.0,EU,UA,ios,232720,organic,None,2021-05-31 12:33:20
1176900,2021-05-31 13:40:42,achieve_level_5,0.0,EU,UA,ios,232720,organic,None,2021-05-31 12:33:20
1176901,2021-05-31 14:15:21,FirstLaunchApp,0.0,EU,RU,ios,232722,organic,None,2021-05-31 14:15:21
1176902,2021-05-31 14:20:52,FirstLaunchApp,0.0,EU,RU,ios,232723,organic,None,2021-05-31 14:20:52


In [ ]:
# Посчитаем столбец разницы столбца event_time от first_log
df3_2['date_diff'] = df3_2['event_time'] - df3_2['first_log']
df3_2

,event_time,event_name,revenue_usd,region,country,platform,user_id,user_type,content_id,first_log,date_diff
0,2020-11-13 13:43:31,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:00:00
1,2020-11-13 13:50:12,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:06:41
2,2020-11-13 13:58:15,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:14:44
3,2020-11-13 14:08:26,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:24:55
4,2020-11-13 14:20:02,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:36:31
...,...,...,...,...,...,...,...,...,...,...,...
1176899,2021-05-31 13:19:12,achieve_level_4,0.0,EU,UA,ios,232720,organic,None,2021-05-31 12:33:20,0 days 00:45:52
1176900,2021-05-31 13:40:42,achieve_level_5,0.0,EU,UA,ios,232720,organic,None,2021-05-31 12:33:20,0 days 01:07:22
1176901,2021-05-31 14:15:21,FirstLaunchApp,0.0,EU,RU,ios,232722,organic,None,2021-05-31 14:15:21,0 days 00:00:00
1176902,2021-05-31 14:20:52,FirstLaunchApp,0.0,EU,RU,ios,232723,organic,None,2021-05-31 14:20:52,0 days 00:00:00


In [ ]:
# date_diff не должен быть отрицательным
df3_2 = df3_2[df3_2['date_diff'] <= '1 days 00:00:00']
df3_2

,event_time,event_name,revenue_usd,region,country,platform,user_id,user_type,content_id,first_log,date_diff
0,2020-11-13 13:43:31,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:00:00
1,2020-11-13 13:50:12,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:06:41
2,2020-11-13 13:58:15,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:14:44
3,2020-11-13 14:08:26,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:24:55
4,2020-11-13 14:20:02,LaunchApp,0.0,EU,RU,android,100002,organic,None,2020-11-13 13:43:31,0 days 00:36:31
...,...,...,...,...,...,...,...,...,...,...,...
1176899,2021-05-31 13:19:12,achieve_level_4,0.0,EU,UA,ios,232720,organic,None,2021-05-31 12:33:20,0 days 00:45:52
1176900,2021-05-31 13:40:42,achieve_level_5,0.0,EU,UA,ios,232720,organic,None,2021-05-31 12:33:20,0 days 01:07:22
1176901,2021-05-31 14:15:21,FirstLaunchApp,0.0,EU,RU,ios,232722,organic,None,2021-05-31 14:15:21,0 days 00:00:00
1176902,2021-05-31 14:20:52,FirstLaunchApp,0.0,EU,RU,ios,232723,organic,None,2021-05-31 14:20:52,0 days 00:00:00


In [ ]:
# Считаем кол-во значений в столбце event_name
df3_2['event_name'].value_counts().to_frame()

,event_name
LaunchApp,90877
FirstLaunchApp,38764
achieve_level_4,14550
achieve_level_5,9915
achieve_level_6,5420
achieve_level_7,2942
af_purchase,2013
achieve_level_8,1472
achieve_level_9,656
achieve_level_10,334


In [ ]:
# Посмотрим на столбец в долях
df3_2['event_name'].value_counts(normalize=True).to_frame()

,event_name
LaunchApp,0.871140
FirstLaunchApp,0.043830
achieve_level_4,0.014900
achieve_level_5,0.011598
af_purchase,0.011328
achieve_level_6,0.008492
achieve_level_7,0.006882
achieve_level_8,0.005705
achieve_level_9,0.004639
achieve_level_10,0.003826


# Визуализиация 3-го задания

In [ ]:
# Создадим новую таблицу путём группировки df3_2 по столбцам region и event_name с уникальными id, сортировку также выполним по этим столбцам
df3_2_reg = df3_2.groupby(['region', 'event_name'], as_index=False)['user_id'].nunique()
df3_2_reg.sort_values(by=['region', 'event_name'])

,region,event_name,user_id
0,AF,FirstLaunchApp,28
1,AF,LaunchApp,22
2,AF,achieve_level_4,9
3,AF,achieve_level_5,4
4,AF,achieve_level_6,2
...,...,...,...
74,SA,achieve_level_5,15
75,SA,achieve_level_6,10
76,SA,achieve_level_7,5
77,SA,achieve_level_8,1


In [ ]:
# Визуализируем df3_2_reg
fig5 = px.bar(df3_2_reg, x='event_name', y='user_id', color = 'region', title='Распределение типа достижений по регионам')
fig5.show(renderer='colab')

In [ ]:
# Создадим новую таблицу путём группировки df3_2 по столбцам country и event_name с уникальными id
df3_2_country = df3_2.groupby(['country', 'event_name'], as_index=False)['user_id'].nunique()
df3_2_country

,country,event_name,user_id
0,AD,FirstLaunchApp,1
1,AE,FirstLaunchApp,6
2,AE,LaunchApp,5
3,AF,FirstLaunchApp,1
4,AF,LaunchApp,3
...,...,...,...
679,ZA,FirstLaunchApp,3
680,ZA,LaunchApp,4
681,ZA,achieve_level_4,2
682,ZA,achieve_level_5,1


In [ ]:
# Визуализируем df3_2_country
fig6 = px.bar(df3_2_country, x='event_name', y='user_id', color = 'country', title='Распределение типа достижений по странам')
fig6.show(renderer='colab')

In [ ]:
# Создадим новую таблицу путём группировки df3_2 по столбцам user_type и event_name с уникальными id
df3_2_user_type = df3_2.groupby(['user_type', 'event_name'], as_index=False)['user_id'].nunique()
df3_2_user_type

,user_type,event_name,user_id
0,non_organic,FirstLaunchApp,6116
1,non_organic,LaunchApp,13842
2,non_organic,achieve_level_10,115
3,non_organic,achieve_level_11,76
4,non_organic,achieve_level_12,60
5,non_organic,achieve_level_13,37
6,non_organic,achieve_level_14,31
7,non_organic,achieve_level_15,27
8,non_organic,achieve_level_16,21
9,non_organic,achieve_level_17,15


In [ ]:
# Визуализируем df3_2_user_type
fig7 = px.bar(df3_2_user_type, x='event_name', y='user_id', color = 'user_type', title='Распределение типа достижений в разрезе траффика')
fig7.show(renderer='colab')

In [ ]:
# Создадим новую таблицу путём группировки df3_2 по столбцам platform и event_name с уникальными id
df3_2_platform = df3_2.groupby(['platform', 'event_name'], as_index=False)['user_id'].nunique()
df3_2_platform

,platform,event_name,user_id
0,android,FirstLaunchApp,20758
1,android,LaunchApp,24201
2,android,achieve_level_10,226
3,android,achieve_level_11,133
4,android,achieve_level_12,95
5,android,achieve_level_13,61
6,android,achieve_level_14,46
7,android,achieve_level_15,35
8,android,achieve_level_16,21
9,android,achieve_level_17,16


In [ ]:
# Визуализируем df3_2_user_type
fig8 = px.bar(df3_2_platform, x='event_name', y='user_id', color = 'platform', title='Распределение типа достижений в разрезе платформ')
fig8.show(renderer='colab')

In [ ]:
# Создадим новую таблицу путём группировки df3_2 по столбцу event_name с уникальными id
df3_2_all = df3_2.groupby(['event_name'], as_index=False)['user_id'].count().sort_values(by='user_id', ascending=False)
df3_2_all

,event_name,user_id
1,LaunchApp,90877
0,FirstLaunchApp,38764
12,achieve_level_4,14550
13,achieve_level_5,9915
14,achieve_level_6,5420
15,achieve_level_7,2942
18,af_purchase,2013
16,achieve_level_8,1472
17,achieve_level_9,656
2,achieve_level_10,334


In [ ]:
# Визуализируем df3_2_user_type
fig8 = px.bar(df3_2_all, x='event_name', y='user_id', title='Распределение по всем типам пользователей')
fig8.show(renderer='colab')

Вывод. Максимальное достижение игрков после запуска доиграть до 10-11 уровня (в порядке исключения). Большинство отваливается уже после 4-го, не смотря на большое кол-вл заупуска приложения. Вероятно проблемы начинаются по завершению обучения. Поведения игроков в первые 24 часа аналогично для разных россмотренных когорт.